# Clinical workflow integration

The imaging data provided for training the model was transformed from DICOM format into .png to help aid in the image pre-processing and model training steps of this project. In the real world, however, the pixel-level imaging data are contained inside of standard DICOM files. 

This notebook creates a DICOM wrapper that takes in a standard DICOM file and outputs data in the format accepted by the model:
* Proper image acquisition type (i.e. X-ray)
* Proper image acquisition orientation (i.e. those present in your training data)
* Proper body part in acquisition

In [1]:
import keras 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom

from PIL import Image
from typing import Optional, Tuple

%matplotlib inline

Using TensorFlow backend.


In [2]:
def read_dicom(filename:str) -> Optional[np.array]:
    '''Reads in a .dcm file and returns an array with the image data
    
    The model returns None if the DICOM file is not appropriate for the
    classifier. In particular, it checks for:
        - Digital radiographies
        - Chest parts
        - Body position 
    
    See: https://www.dicomlibrary.com/dicom/ for code references
    '''
    ds = pydicom.dcmread(filename)
    
    if ds.Modality != 'DX':
        print('Modality not supported:', ds.Modality)
        print('This device only works with the DX modality (Digital Radiography)')
        return None
    
    if ds.BodyPartExamined != 'CHEST':
        print('Body part not supported:', ds.BodyPartExamined)
        print('This device only works with Chest Radiographies')
        return None
    
    if ds.PatientPosition not in ['AP', 'PA']:
        print('Body position not supported:', ds.PatientPosition)
        print('This device only works with AP or PA positions')
        return None
    
    print(f'Patient age: {ds.PatientAge}, sex: {ds.PatientSex}')
    print('Study description:', ds.StudyDescription)
    return ds.pixel_array
    
    
def preprocess_image(img_array : np.array, img_size : Tuple[int]):
    '''Pre-process the image to the format expected by the classifier'''
    batch, height, width, channels = img_size
    img = Image.fromarray(img_array)
    img = img.resize((height, width))
    img = np.array(img)
    img = img / 255.
    
    if len(img.shape) == 2:
        img = np.dstack((img, img, img))
    img = np.expand_dims(img, axis=0)
    return img


def load_model(model_path, weight_path):
    '''Loads in a model with weights and compiles it'''
    with open(model_path) as model_json:
        model = keras.models.model_from_json(model_json.read())
    model.load_weights(weight_path)
    return model


def predict_image(model, img, thresh): 
    '''Predict the presence of pneumonia in the image'''
    prob = model.predict(img)[0][0]
    print('Probability of Pneumonia:', prob)
    prediction = prob > thresh
    print('Predicted Pneumonia:', prediction)
    print()
    return prediction

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json'
weight_path = 'xray_class_my_model.best.hdf5'

IMG_SIZE=(1, 224, 224, 3)

my_model = load_model(model_path, weight_path)
thresh = 0.13216244

# use the .dcm files to test your prediction
for i in test_dicoms:
    img = read_dicom(i)
    if img is None:
        continue
        
    img_proc = preprocess_image(img, IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)

Patient age: 81, sex: M
Study description: No Finding
Probability of Pneumonia: 0.67513233
Predicted Pneumonia: True

Patient age: 58, sex: M
Study description: Cardiomegaly
Probability of Pneumonia: 0.055348597
Predicted Pneumonia: False

Patient age: 77, sex: M
Study description: Effusion
Probability of Pneumonia: 0.22254159
Predicted Pneumonia: True

Body part not supported: RIBCAGE
This device only works with Chest Radiographies
Modality not supported: CT
This device only works with the DX modality (Digital Radiography)
Body position not supported: XX
This device only works with AP or PA positions
